In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.alert import Alert
import base64
from PIL import Image
from io import BytesIO
import pytesseract
import string
import pandas as pd
import time
import numpy as np
import cv2

# Path to the Tesseract executable (change it according to your installation)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [1]:
def code(l,y,s,d,rad):
    select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "court_complex_code")))
    select = Select(select_element)
    select.select_by_value(l)
    input_element1 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "petres_name")))
    input_element1.clear()  # Clear any existing text in the input field
    input_element1.send_keys("GST")
    input_element2 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "rgyear")))
    input_element2.clear()  # Clear any existing text in the input field
    input_element2.send_keys(str(y))
    driver.find_element(By.ID,rad).click()
    t=driver.find_element(By.ID,"captcha_image")
    image=t.screenshot_as_base64
    captcha_screenshot_data = base64.b64decode(image)
    img = Image.open(BytesIO(captcha_screenshot_data))
    img.save("C:/Users/ajayv/captcha.png")
    result=captcha_recognition()
    driver.find_element(By.ID,"captcha").send_keys(result)
    button = driver.find_element(By.CSS_SELECTOR,'input[type="button"][name="submit1"]')
    button.click()
    try:
        alert = WebDriverWait(driver, 20).until(EC.alert_is_present())
        # Alert is present
        # Perform desired actions here
        alert_text = alert.text
        #print(alert_text)
        # Accept the alert
        alert.accept()
        driver.refresh()
        code(l,y,s,d,rad)
    except :
        pass
    try:
        error_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "errSpan")))
        error_message = error_element.text
        if error_message=="Invalid Captcha":
            #printf("Invalid Captcha")
            code(l,y,s,d,rad)  
        else:
            pass
    except:
        pass  

In [ ]:
def captcha_recognition():
    image_path = 'C:/Users/ajayv/captcha.png'
    # Open the image using PIL
    image = Image.open(image_path)
    # Perform OCR using Tesseract
    text = pytesseract.image_to_string(image,lang="eng")
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Print the extracted text
    return text.lower()

In [ ]:
year=[]
case=[]
petitioner=[]
links=[] 
state_code=[]
district_code=[]
court=[]
rads=[]

In [ ]:
driver = webdriver.Chrome() 

In [ ]:
s=2
for d in [15,17,19,20]:
    try:
        driver.get(f'https://services.ecourts.gov.in/ecourtindia_v4_bilingual/cases/ki_petres.php?state=D&state_cd={s}&dist_cd={d}')
    except:
        break
    select_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "court_complex_code")))
    options = select_element.find_elements(By.TAG_NAME, "option")
    lst=[]
    option_text=[]
    for option in options:
        option_value = option.get_attribute("value")
        option_text.append(option.text)
        lst.append(option_value)
    lst=lst[1:]
    option_text=option_text[1:]
    for y in range(2017,2022):
        for l in range(len(lst)):
            for rad in ["radD","radP"]:
                code(lst[l],y,s,d,rad)
                table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "showList1")))

                # Find all rows in the table
                rows = table.find_elements(By.TAG_NAME,"tr")

                # Iterate over each row and extract the data
                for row in rows:
                    cells = row.find_elements(By.TAG_NAME,"td")
                    try:
                        if len(cells)>1:
                            sr_no = cells[0].text
                            case_details = cells[1].text
                            petitioner_respondent = cells[2].text
                            view_link = cells[3].find_element(By.TAG_NAME,"a").get_attribute("href")

                            # Print or process the extracted data as per your requirement
                            rads.append(rad)
                            state_code.append(s)
                            district_code.append(d)
                            court.append(option_text[l])
                            year.append(y)                   
                            case.append(case_details)
                            petitioner.append(petitioner_respondent)
                            links.append(view_link)
                        else:
                            pass
                    except:
                        pass
        print(f"records in year {y} are {len(state_code)} for district {d}")

In [ ]:
d=pd.DataFrame()
d["state_code"]=state_code
d["district_code"]=district_code
d["court"]=court
d["year"]=year
d["case"]=case
d["petitioner"]=petitioner
d["links"]=links
d["Pending/Disposed"]=rads
d.dropna()